In [111]:
# !pip install selenium

### Import Libraries

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

### Creating Webdriver *for* Browser Automation

In [13]:
driver = webdriver.Firefox()
# driver = webdriver.Chrome()

In [58]:
# driver.get("https://election.gov.np/np/page/voter-list-db")
driver.get("https://voterlist.election.gov.np/bbvrs1/index_2.php")

# driver.get("https://www.selenium.dev/selenium/web/web-form.html")

In [59]:
title = driver.title
print(title)

निर्वाचन आयोग अन्तिम मतदाता नामावली


In [60]:
driver.implicitly_wait(0.5)

### Select Dropdowns and a Submit Button via XPATH

In [61]:
# element = driver.find_element(By.XPATH, '//div[@class="margin-top-15"]')
# element2 = element.find_element(By.XPATH, '//div[@class="panel-group"]')

state_dropdown = driver.find_element(By.XPATH, '//select[@id="state"]')
dist_dropdown = driver.find_element(By.XPATH, '//select[@id="district"]')
mun_dropdown = driver.find_element(By.XPATH, '//select[@id="vdc_mun"]')
ward_dropdown = driver.find_element(By.XPATH, '//select[@id="ward"]')
reg_centre_dropdown = driver.find_element(By.XPATH, '//select[@id="reg_centre"]')
submit_button = driver.find_element(By.ID, "btnSubmit")

### Select the First Value of Each Dropdown 

In [62]:
from selenium.webdriver.support.select import Select
Select(state_dropdown).select_by_index(1)
Select(dist_dropdown).select_by_index(1)
Select(mun_dropdown).select_by_index(1)
Select(ward_dropdown).select_by_index(1)
Select(reg_centre_dropdown).select_by_index(1)
submit_button.click()

In [63]:
# print(driver.current_url)

### Change the default 10 rows to 100 rows in a page

In [64]:

entries_dropdown = driver.find_element(By.XPATH, '//select[@name="tbl_data_length"]')
Select(entries_dropdown).select_by_value("100")

In [12]:
# !pip install lxml

In [13]:
# !pip install html5lib

In [14]:
# driver.page_source

In [15]:
# from bs4 import BeautifulSoup
# import pandas as pd
# from io import StringIO

# page_content = driver.page_source
# soup = BeautifulSoup(page_content, 'html.parser')
# table = soup.find('table', class_='bbvrs_data dataTable')
# df = pd.read_html(StringIO(str(table)))[0]
# df
# # df.to_csv('table1.csv', index=False)

In [16]:
# select_next = driver.find_element(By.ID, "tbl_data_next")
# select_next.click()

### Scrape all the table until no more pages is available

In [18]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from selenium.common.exceptions import NoSuchElementException

df = pd.DataFrame()

def scraper():
    global df  
    
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find('table', class_='bbvrs_data dataTable')
    data = pd.read_html(StringIO(str(table)))[0]
    
    df = pd.concat([df, data], ignore_index=True)

while True:
    # execute the scraper function
    scraper()

    try:
        # find and click the next page link
        next_page_link = driver.find_element(By.CLASS_NAME, "paginate_enabled_next")

        if next_page_link:
            driver.execute_script("arguments[0].click();", next_page_link)
            # next_page_link.click()
    except NoSuchElementException:
        print("No more pages available")
        break

No more pages available


### Dataframe / Dataset of voters of a registration center

In [21]:
df

,सि.नं.,मतदाता नं,मतदाताको नाम,उमेर (वर्ष),लिङ्ग,पति/पत्नीको नाम,पिता/माताको नाम,विवरण
0,1,8008177,अगम सिंह लिम्बु,75,पुरुष,टिका माया,राज बहादुर/ धन माया,NaN
1,2,22005851,अनमोल तामाङ्ग,27,पुरुष,-,दावा प्रसाद/ बुद्धि माया,NaN
2,3,22112867,अनिता नेम्बाङ,28,महिला,खेम राज,नारायण दास/ दुर्गा माया,NaN
3,4,8007799,अनिता लिम्बु,56,महिला,बिरेन्द्र कुमार,मतन बहादुर/ भिम माया,NaN
4,5,14112798,अनिल गुरुङ्ग,29,पुरुष,-,खड्ग बहादुर/ सावित्री,NaN
...,...,...,...,...,...,...,...,...
783,784,8007821,हेमा जोगी,36,महिला,शिव कुमार,बबुर लाल/ तारा देवी,NaN
784,785,8007843,होम कुमारी गुरुङ्ग,73,महिला,खम्व सिहं,तीर्थ मान/ मङ्गल माया,NaN
785,786,8008034,होम कुमारी चौहान,71,महिला,तारा बहादुर,अम्बर बहादुर/ टंक माया,NaN
786,787,16608254,होम वहादुर खवास,36,पुरुष,गिता,डम्‍वर वहादुर/ इन्‍दिरा,NaN


### Function to save df to table in MySQL database

Improvements
- Use of helper function for saving table
- Store the passwords in .env file

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import URL


def df_table(dataframe, database_name, table_name):
    
    conn_url = URL.create(
        "mysql+mysqlconnector",
        username = "root",
        password = "mysql@123",
        host = "localhost",
        port = 3306,
        database = database_name) 
    
    engine = create_engine(conn_url)
    con = engine.connect()
    dataframe.to_sql(table_name, con=con, if_exists='replace', index=False)  

In [24]:
df_table(df, 'voter', 'output')

In [150]:
driver.quit()

In [20]:
# df.to_csv('output.csv', encoding='utf-8', index=False)

In [159]:
# !pip install xlrd

In [160]:
# !pip install openpyxl

In [161]:
# df.to_excel('output.xlsx', index = False, sheet_name = 'NewSheet')

### Functionality to go back to home page when no next page is available

In [25]:
go_back = driver.find_element(By.CLASS_NAME, "a_back")
go_back.click()

### Putting it all together

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

# Initialize WebDriver
driver = webdriver.Firefox()
driver.get("https://voterlist.election.gov.np/bbvrs1/index_2.php")
driver.implicitly_wait(0.2)

df = pd.DataFrame()

def post_submit_actions():
    entries_dropdown = driver.find_element(By.XPATH, '//select[@name="tbl_data_length"]')
    Select(entries_dropdown).select_by_value("100")

    def scraper():
        global df  
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        table = soup.find('table', class_='bbvrs_data dataTable')
        data = pd.read_html(StringIO(str(table)))[0]       
        df = pd.concat([df, data], ignore_index=True)

    while True:
        scraper()

        try:
            next_page_link = driver.find_element(By.CLASS_NAME, "paginate_enabled_next")

            if next_page_link:
                driver.execute_script("arguments[0].click();", next_page_link)
                # next_page_link.click()
        except NoSuchElementException:
            print("No more pages available")
            break

# Loop through each combination 
for state_index in range(4, 5):  #(1,7+1) # Maximum possible values
    try:
        for dist_index in range(1, 12): #(1, 14+1)
            try:
                for mun_index in range(1, 15): #(1, 18+1)
                    try:
                        for ward_index in range(1, 33+1): #(1, 33+1)
                            try:
                                for reg_centre_index in range(1, 9+1): #(1, 9+1)
                                    try:
                                        state_dropdown = driver.find_element(By.XPATH, '//select[@id="state"]')
                                        dist_dropdown = driver.find_element(By.XPATH, '//select[@id="district"]')
                                        mun_dropdown = driver.find_element(By.XPATH, '//select[@id="vdc_mun"]')
                                        ward_dropdown = driver.find_element(By.XPATH, '//select[@id="ward"]')
                                        reg_centre_dropdown = driver.find_element(By.XPATH, '//select[@id="reg_centre"]')
                                        submit_button = driver.find_element(By.ID, "btnSubmit")

                                        Select(state_dropdown).select_by_index(state_index)
                                        Select(dist_dropdown).select_by_index(dist_index)
                                        Select(mun_dropdown).select_by_index(mun_index)
                                        Select(ward_dropdown).select_by_index(ward_index)
                                        Select(reg_centre_dropdown).select_by_index(reg_centre_index)
                                        submit_button.click()

                                        post_submit_actions()
                                        
                                        go_back = driver.find_element(By.CLASS_NAME, "a_back")
                                        go_back.click()
                                        
                                    except NoSuchElementException:
                                        print("Element not found. Skipping...")
                            except NoSuchElementException:
                                break
                    except NoSuchElementException:
                        break
            except NoSuchElementException:
                break
    except NoSuchElementException:
        break
# driver.quit()


In [14]:
# df.to_csv('gandaki.csv',  header=True,  index=False)

### Few improvements

- No repetion of Voter ID 
- Modified entries_dropdown from 10 to 1000 to scrap in single attempt but no significant improvement in speed 

In [89]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

# Initialize WebDriver
driver = webdriver.Firefox()
driver.get("https://voterlist.election.gov.np/bbvrs1/index_2.php")
driver.implicitly_wait(0.2)

df = pd.DataFrame()

def scraper():
    global df 
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find('table', class_='bbvrs_data dataTable')
    converters = {'मतदाता नं': str}
    data = pd.read_html(StringIO(str(table)), converters=converters)[0]
    df = pd.concat([df, data], ignore_index=True)

def post_submit_actions():
    driver.execute_script("document.querySelector('select[name=\"tbl_data_length\"] option[value=\"100\"]').value = '10000';")
    entries_dropdown = driver.find_element(By.XPATH, '//select[@name="tbl_data_length"]')
    Select(entries_dropdown).select_by_value("10000")

    scraper()

# Loop through each combination 
for state_index in range(2, 3):  #(1,7+1)
    try:
        for dist_index in range(1, 9): #(1, 14+1)
            try:
                for mun_index in range(1, 19): #(1, 18+1)
                    try:
                        for ward_index in range(1, 26): #(1, 33+1)
                            try:
                                for reg_centre_index in range(1, 10): #(1, 9+1)
                                    try:
                                        state_dropdown = driver.find_element(By.XPATH, '//select[@id="state"]')
                                        dist_dropdown = driver.find_element(By.XPATH, '//select[@id="district"]')
                                        mun_dropdown = driver.find_element(By.XPATH, '//select[@id="vdc_mun"]')
                                        ward_dropdown = driver.find_element(By.XPATH, '//select[@id="ward"]')
                                        reg_centre_dropdown = driver.find_element(By.XPATH, '//select[@id="reg_centre"]')
                                        submit_button = driver.find_element(By.ID, "btnSubmit")

                                        Select(state_dropdown).select_by_index(state_index)
                                        Select(dist_dropdown).select_by_index(dist_index)
                                        Select(mun_dropdown).select_by_index(mun_index)
                                        Select(ward_dropdown).select_by_index(ward_index)
                                        Select(reg_centre_dropdown).select_by_index(reg_centre_index)
                                        submit_button.click()

                                        post_submit_actions()
                                        
                                        go_back = driver.find_element(By.CLASS_NAME, "a_back")
                                        go_back.click()
                                        
                                    except NoSuchElementException:
                                        break
                            except NoSuchElementException:
                                break
                    except NoSuchElementException:
                        break
            except NoSuchElementException:
                break
    except NoSuchElementException:
        break

# driver.quit()
